# Google Firestore (Native 模式)

> [Firestore](https://cloud.google.com/firestore) 是一款无服务器文档导向数据库，可以根据任何需求进行扩展。利用 Firestore 的 Langchain 集成，扩展您的数据库应用程序以构建支持 AI 的体验。

本 Notebook 将介绍如何使用 `FirestoreLoader` 和 `FirestoreSaver` 将 [Firestore](https://cloud.google.com/firestore) 用于 [保存、加载和删除 langchain 文档](/docs/how_to#document-loaders)。

在 [GitHub](https://github.com/googleapis/langchain-google-firestore-python/) 上了解有关此软件包的更多信息。

[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-firestore-python/blob/main/docs/document_loader.ipynb)

## 开始之前

要运行此笔记本，您需要执行以下操作：

* [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
* [启用 Firestore API](https://console.cloud.google.com/flows/enableapi?apiid=firestore.googleapis.com)
* [创建 Firestore 数据库](https://cloud.google.com/firestore/docs/manage-databases)

在笔记本的运行时环境中确认数据库访问权限后，请填写以下值并运行单元格，然后再运行示例脚本。

In [ ]:
# @markdown Please specify a source for demo purpose.
SOURCE = "test"  # @param {type:"Query"|"CollectionGroup"|"DocumentReference"|"string"}

### 🦜🔗 库安装

集成位于其自有的 `langchain-google-firestore` 包中，所以我们需要安装它。

In [ ]:
%pip install --upgrade --quiet langchain-google-firestore

**Colab 专属**: 取消注释下面的代码单元格以重启内核，或使用按钮重启内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重启终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便您可以在此笔记本中使用 Google Cloud 资源。

如果您不知道您的项目 ID，可以尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 请参阅支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 🔐 身份验证

通过笔记本中的登录 IAM 用户验证到 Google Cloud，以便访问你的 Google Cloud 项目。

- 如果你正在使用 Colab 运行此笔记本，请使用下面的代码单元格并继续。
- 如果你正在使用 Vertex AI Workbench，请在此处查看设置说明：[https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [ ]:
from google.colab import auth

auth.authenticate_user()

## 基本用法

### 保存文档

`FirestoreSaver` 可以将 Documents 存储到 Firestore 中。默认情况下，它会尝试从元数据中提取 Document 引用。

使用 `FirestoreSaver.upsert_documents(<documents>)` 保存 Langchain 文档。

In [ ]:
from langchain_core.documents import Document
from langchain_google_firestore import FirestoreSaver

saver = FirestoreSaver()

data = [Document(page_content="Hello, World!")]

saver.upsert_documents(data)

#### 保存无引用的文档

如果指定了 collection，文档将使用自动生成的 id 进行存储。

In [ ]:
saver = FirestoreSaver("Collection")

saver.upsert_documents(data)

#### 将文档与其它参考资料一同保存

In [ ]:
doc_ids = ["AnotherCollection/doc_id", "foo/bar"]
saver = FirestoreSaver()

saver.upsert_documents(documents=data, document_ids=doc_ids)

### 从集合或子集合加载

使用 `FirestoreLoader.load()` 或 `Firestore.lazy_load()` 加载 langchain 文档。`lazy_load` 返回一个在迭代期间才查询数据库的生成器。要初始化 `FirestoreLoader` 类，你需要提供：

1. `source` - 一个 `Query`、`CollectionGroup`、`DocumentReference` 实例，或者一个 Firestore 集合的单斜杠 `\` 分隔路径。

In [ ]:
from langchain_google_firestore import FirestoreLoader

loader_collection = FirestoreLoader("Collection")
loader_subcollection = FirestoreLoader("Collection/doc/SubCollection")


data_collection = loader_collection.load()
data_subcollection = loader_subcollection.load()

### 加载单个文档

In [ ]:
from google.cloud import firestore

client = firestore.Client()
doc_ref = client.collection("foo").document("bar")

loader_document = FirestoreLoader(doc_ref)

data = loader_document.load()

### 从 CollectionGroup 或 Query 加载

In [ ]:
from google.cloud.firestore import CollectionGroup, FieldFilter, Query

col_ref = client.collection("col_group")
collection_group = CollectionGroup(col_ref)

loader_group = FirestoreLoader(collection_group)

col_ref = client.collection("collection")
query = col_ref.where(filter=FieldFilter("region", "==", "west_coast"))

loader_query = FirestoreLoader(query)

### 删除文档

使用 `FirestoreSaver.delete_documents(<documents>)` 从 Firestore 集合中删除一组 langchain 文档。

如果提供了文档 ID，文档将被忽略。

In [ ]:
saver = FirestoreSaver()

saver.delete_documents(data)

# The Documents will be ignored and only the document ids will be used.
saver.delete_documents(data, doc_ids)

## 高级用法

### 使用自定义文档页面内容和元数据加载文档

`page_content_fields` 和 `metadata_fields` 的参数将指定写入 LangChain Document 的 `page_content` 和 `metadata` 的 Firestore Document 字段。

In [ ]:
loader = FirestoreLoader(
    source="foo/bar/subcol",
    page_content_fields=["data_field"],
    metadata_fields=["metadata_field"],
)

data = loader.load()

#### 定制页面内容格式

当 `page_content` 只包含一个字段时，信息将仅显示该字段的值。否则，`page_content` 将采用 JSON 格式。

### 自定义连接和身份验证

In [ ]:
from google.auth import compute_engine
from google.cloud.firestore import Client

client = Client(database="non-default-db", creds=compute_engine.Credentials())
loader = FirestoreLoader(
    source="foo",
    client=client,
)